In [2]:
path='C:/Users/student/Downloads/데이터들/'

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.optimizers import *
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

from nltk.tokenize import sent_tokenize
from konlpy.corpus import kolaw
from konlpy.tag import Okt

Using TensorFlow backend.


In [3]:
chatbotData=pd.read_csv(path+'ChatData.csv')
question, answer = list(chatbotData['Q']), list(chatbotData['A'])

In [4]:
len(answer)

11823

In [5]:
c=kolaw.open('constitution.txt').read()

In [6]:
sent_tokenize(c) # 문장단위로 

['대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.',
 '제1장 총강\n  제1조 ① 대한민국은 민주공화국이다.',
 '②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.',
 '제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.',
 '②국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.',
 '제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.',
 '제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.',
 '제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.',
 '②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.',
 '제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.',
 '②외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.',
 '제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.',
 '②공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 

In [7]:
okt=Okt()
okt.pos("오늘은 날씨가 따뜻합니다")

[('오늘', 'Noun'),
 ('은', 'Josa'),
 ('날씨', 'Noun'),
 ('가', 'Josa'),
 ('따뜻합니다', 'Adjective')]

In [8]:
doc0=[" ".join([''.join(w) for w, t in okt.pos(s) if t not in ['Number','Foreign'] and w not in ['제','조']]) for s in sent_tokenize(c)]

In [9]:
tokenizer=Tokenizer()
tokenizer.word_index

{}

In [10]:
tokenizer.fit_on_texts(doc0)
doc=[tok for tok in tokenizer.texts_to_sequences(doc0) if len(tok)>1]

In [11]:
maxLen=max([len(x)-1 for x in doc])
print(maxLen)
vocabSize=len(tokenizer.word_index)+1
print(vocabSize)

187
1165


In [12]:
sent=[s for s in sent_tokenize(c)]
sent[0]

'대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.'

In [13]:
sent_tokenize(c)

['대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.',
 '제1장 총강\n  제1조 ① 대한민국은 민주공화국이다.',
 '②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.',
 '제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.',
 '②국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.',
 '제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.',
 '제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.',
 '제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.',
 '②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.',
 '제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.',
 '②외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.',
 '제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.',
 '②공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 

In [14]:
# konlpy.rog => API => Okt
temp=sent_tokenize(c)[0:5]
for i in range(10):
    for w,t in okt.pos(temp[i]): # 형태소로 분리하는 함수
        print('w:',w)
        print("t:",t)

w: 대한민국
t: Noun
w: 헌법
t: Noun
w: 


t: Foreign
w: 유구
t: Noun
w: 한
t: Josa
w: 역사
t: Noun
w: 와
t: Josa
w: 전통
t: Noun
w: 에
t: Josa
w: 빛나는
t: Verb
w: 우리
t: Noun
w: 대
t: Modifier
w: 한
t: Modifier
w: 국민
t: Noun
w: 은
t: Josa
w: 3
t: Number
w: ·
t: Punctuation
w: 1
t: Number
w: 운동
t: Noun
w: 으로
t: Josa
w: 건립
t: Noun
w: 된
t: Verb
w: 대한민국
t: Noun
w: 임시정부
t: Noun
w: 의
t: Josa
w: 법
t: Noun
w: 통과
t: Noun
w: 불의
t: Noun
w: 에
t: Josa
w: 항거
t: Noun
w: 한
t: Josa
w: 4
t: Number
w: ·
t: Punctuation
w: 19
t: Number
w: 민주
t: Noun
w: 이념
t: Noun
w: 을
t: Josa
w: 계승
t: Noun
w: 하고
t: Josa
w: ,
t: Punctuation
w: 조국
t: Noun
w: 의
t: Josa
w: 민주
t: Noun
w: 개혁
t: Noun
w: 과
t: Josa
w: 평화
t: Noun
w: 적
t: Suffix
w: 통일
t: Noun
w: 의
t: Josa
w: 사명
t: Noun
w: 에
t: Josa
w: 입
t: Noun
w: 각하
t: Noun
w: 여
t: Josa
w: 정의
t: Noun
w: ·
t: Punctuation
w: 인도
t: Noun
w: 와
t: Josa
w: 동포
t: Noun
w: 애
t: Noun
w: 로써
t: Noun
w: 민족
t: Noun
w: 의
t: Josa
w: 단결
t: Noun
w: 을
t: Josa
w: 공고
t: Noun
w: 히
t: Adverb
w: 하고
t: Verb
w: ,
t: Punctuation
w

IndexError: list index out of range

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(doc0)

In [ ]:
len(tokenizer.index_word)
tokenizer.word_index

In [ ]:
tokenizer.texts_to_sequences(doc0)

In [ ]:
np.shape(tokenizer.texts_to_sequences(doc0))

In [ ]:
# for a in tokenizer.texts_to_sequences(doc0):
#     if len(a)<=1:
#         print(len(a))
doc=[tok for tok in tokenizer.texts_to_sequences(doc0) if len(tok)>1]     


In [ ]:
#maxLen=max([len(x) for x in doc])
#tokenizer.word_counts    
vocabSize=len(tokenizer.word_index)+1
print(maxLen, vocabSize) # 187, 1165

In [ ]:
doc

In [ ]:
def genData(doc, maxLen, vocabSize):
    for sent in doc:
        inputs=[]
        targets=[]
        for i in range(1, len(sent)): # 1~6
            inputs.append(sent[0:i]) # [[45], [45,139] ...
            targets.append(sent[i])  # [439], [648] ...
        y=np_utils.to_categorical(targets, vocabSize)
        inputSeq=sequence.pad_sequences(inputs, maxlen=maxLen)
        yield(inputSeq,y) # 객체가 리턴 됨
#         print("inputs:",inputs)
#         print("-"*50)
#         print("targets:",targets)
#         print("="*50)

In [ ]:
for i, (x,y) in enumerate(genData(doc, maxLen, vocabSize)):
    print(i)
    print("x",x.shape, "\n",x)
    print("y",y.shape, '\n',y)

In [ ]:
def cf():
    for i in range(3):
        yield i*i
        
print(cf())

In [ ]:
genData(doc, maxLen, vocabSize)

In [ ]:
xdata=[]
ydata=[]
for x,y in genData(doc, maxLen, vocabSize):
    xdata.append(x)
    ydata.append(y)
xdata=np.concatenate(xdata)
ydata=np.concatenate(ydata)

In [ ]:
# xdata
# 배열 합치기
a=np.array([[1,2],[3,4]])
b=np.array([[5,6],[7,8],[9,10]])
np.concatenate((a,b), axis=0)
np.concatenate((a,b), axis=None)

In [ ]:
ydata.shape

In [ ]:
model=Sequential()
model.add(Embedding(vocabSize, 100, input_length=maxLen)) # 1165,100
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0,5))
model.add(Dense(vocabSize, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [66]:
hist=model.fit(xdata, ydata, epochs=500, batch_size=800)

Epoch 1/500
6917/6917 [==============================] - 16s 2ms/step - loss: 6.1713 - accuracy: 0.0259
Epoch 2/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.7396 - accuracy: 0.0432
Epoch 3/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6816 - accuracy: 0.0538
Epoch 4/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6694 - accuracy: 0.0538
Epoch 5/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6604 - accuracy: 0.0538
Epoch 6/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6471 - accuracy: 0.0538
Epoch 7/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6242 - accuracy: 0.0538
Epoch 8/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.5925 - accuracy: 0.0538
Epoch 9/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.5556 - accuracy: 0.0548
Epoch 10/500
6917/6917 [==============================] - 17s 2m

6917/6917 [==============================] - 17s 2ms/step - loss: 3.0194 - accuracy: 0.3742
Epoch 80/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.9947 - accuracy: 0.3789
Epoch 81/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.9649 - accuracy: 0.3846
Epoch 82/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.9403 - accuracy: 0.3869
Epoch 83/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.9105 - accuracy: 0.3903
Epoch 84/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.8879 - accuracy: 0.3966
Epoch 85/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.8567 - accuracy: 0.4010
Epoch 86/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.8358 - accuracy: 0.4039
Epoch 87/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.8071 - accuracy: 0.4106
Epoch 88/500
6917/6917 [==============================] - 17s 2ms/st

6917/6917 [==============================] - 17s 2ms/step - loss: 0.6815 - accuracy: 0.8726
Epoch 234/500
6917/6917 [==============================] - 17s 2ms/step - loss: 0.6764 - accuracy: 0.8736
Epoch 235/500
6917/6917 [==============================] - 17s 2ms/step - loss: 0.6683 - accuracy: 0.8741
Epoch 236/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.6649 - accuracy: 0.8767
Epoch 237/500
6917/6917 [==============================] - 17s 2ms/step - loss: 0.6554 - accuracy: 0.8788
Epoch 238/500
6917/6917 [==============================] - 17s 2ms/step - loss: 0.6484 - accuracy: 0.8803
Epoch 239/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.6468 - accuracy: 0.8786
Epoch 240/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.6369 - accuracy: 0.8815
Epoch 241/500
6917/6917 [==============================] - 17s 2ms/step - loss: 0.6336 - accuracy: 0.8819
Epoch 242/500
6917/6917 [==============================] - 1

6917/6917 [==============================] - 17s 3ms/step - loss: 0.1911 - accuracy: 0.9540
Epoch 388/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1916 - accuracy: 0.9529
Epoch 389/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1886 - accuracy: 0.9539
Epoch 390/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1886 - accuracy: 0.9545
Epoch 391/500
6917/6917 [==============================] - 18s 3ms/step - loss: 0.1870 - accuracy: 0.9539
Epoch 392/500
6917/6917 [==============================] - 18s 3ms/step - loss: 0.1855 - accuracy: 0.9540
Epoch 393/500
6917/6917 [==============================] - 18s 3ms/step - loss: 0.1851 - accuracy: 0.9545
Epoch 394/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1841 - accuracy: 0.9543
Epoch 395/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1831 - accuracy: 0.9542
Epoch 396/500
6917/6917 [==============================] - 1

In [67]:
import matplotlib.pyplot as plt
plt.plot(hist.history['accuracy'])
plt.show()
model.save('tentGen.hdf5')

<Figure size 640x480 with 1 Axes>

In [15]:
from keras.models import load_model
model = load_model('tentGen.hdf5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [91]:
wordList="대한민국 의 국민 이 되는 요건 은 법률 로 정한 다 ."

In [92]:
reverseWordMap=dict(map(reversed,tokenizer.word_index.items()))
len(reverseWordMap)

1164

In [93]:
x=sequence.pad_sequences([[tokenizer.word_index[w] for w in wordList[:2]]], maxlen=maxLen)
x

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [64]:
x=np.array([2.5, 1.5, 0, 1])
x.argsort() # 2, 3, 1, 0
np.flip(x.argsort()) # 내림차순 정렬

array([0, 1, 3, 2], dtype=int64)

In [78]:
p=model.predict(x)
idx=np.flip(np.argsort(p),0)
idx

array([[945, 187, 204, ...,  65,  28,  22]], dtype=int64)

In [ ]:
for i in idx[0]:
    print(reverseWordMap)

In [94]:
def predictWord(i, n): 
    x=sequence.pad_sequences([[tokenizer.word_index[w] for w in wordList[:i]]], maxlen=maxLen)
    p=model.predict(x)[0]
    idx=np.flip(np.argsort(p),0)
    for j in idx[:n]:
        print(" ".join(wordList[:i]), reverseWordMap[j], "(p={:4.2f}%)".format(100*p[j]))

In [95]:
predictWord(1,3) # 대한민국 다음 단어?, 3개 출력
print("="*50)
predictWord(2,3) # 대한민국 의 다음 단어? 3개 출력
print("="*50)
predictWord(5,3) # 대한민국 의 국민 이 되는 다음 3개 출력

대 헌법 (p=35.69%)
대 는 (p=34.92%)
대 국민 (p=18.11%)
대 한 국민 (p=63.13%)
대 한 헌법 (p=14.83%)
대 한 조직 (p=4.92%)


KeyError: '민'